# Utilidades

In [ ]:
from pyspark.sql import functions as f

In [ ]:
#Scroll horizontal en tablas

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Código

## Lectura

In [ ]:
# Lectura del fichero persistido en el sandbox

df=spark.read.parquet("/data/sandboxes/...")

In [ ]:
# Comprobación de la lectura correcta del fichero persistido

df.show(5, False)

## Creación de variables

###  Particular_o_Autonomo 

In [ ]:
# Creación de variable *** Particular_o_Autonomo ***;

df=df\
.withColumn('Tipo_cliente', f.when((f.col('work_contract_type')=="1"), 'A')\
            .otherwise('P'))

In [ ]:
# Comprobración varible *** Particular_o_Autonomo ***;

df\
.select(df.work_contract_type, df.Tipo_cliente)\
.filter(df.work_contract_type == '3').show(5, False)

###  CNAE / CNO, para los autonomos viene informado en el campo CNAE/CNO, en el resto en el campo de actividad economica

In [ ]:
# Creación de variable cnae_cno;

df=df\
.withColumn('cnae_cno', f.when((f.col('work_contract_type')=="1"), df.cnae_cno_id)\
            .otherwise(df.company_cnae_id))

In [ ]:
# Comprobración varible cnae_cno;

df\
.select(df.work_contract_type, df.cnae_cno, df.cnae_cno_id, df.company_cnae_id)\
.filter(df.work_contract_type == '2').show(5, False)

### Codigo destino - subdestino


In [ ]:
# Creación de la variable /* Codigo destino - subdestino */

df=df\
.withColumn('Destino_subdestino', f.when((f.col('ppty_dest_id')==0) | (f.col('ppty_subdest_id')==0), "")\
          .otherwise(f.concat(f.col('ppty_dest_id').cast("string"), f.lit("/"),(f.col('ppty_subdest_id').cast("string")))))

In [ ]:
# Comprobación de la variable /* Codigo destino - subdestino */

df\
.select(df.ppty_dest_id, df.ppty_subdest_id,df.Destino_subdestino)\
.filter("ppty_dest_id == 900")\
.show(5, False)

### Ratio financiacion 

In [ ]:
# Creación de la variable /* Codigo destino - subdestino */

df=df\
.withColumn('ratio_financiacion', f.when((f.col('ppal_good_finance_amount')==0), '')\
           .otherwise(f.col('risk_financed_amount')/f.col('ppal_good_finance_amount')))

In [ ]:
# Comprobración de la variable /* Codigo destino - subdestino */

df\
.select("ppal_good_finance_amount","risk_financed_amount","ratio_financiacion")\
.filter("ppal_good_finance_amount <> 0").show(20, False)

### Ratio financiacion - Propuesta negocio

In [ ]:
# Creación de la variable /* Ratio financiacion - Propuesta negocio*/

df=df\
.withColumn('ratio_financiacion_2', f.when((f.col('ppal_good_finance_amount')==0), '')\
           .otherwise(f.col('prop_lim_operation_amount')/f.col('ppal_good_finance_amount')))

In [ ]:
# Comprobación de la variable /* Ratio financiacion - Propuesta negocio*/

df\
.select("ppal_good_finance_amount", "prop_lim_operation_amount", "ratio_financiacion_2")\
.filter('ppal_good_finance_amount == 0').show(5, False)

### Ratio importe financiado / plazo

In [ ]:
# Creación de la variable /* Ratio importe financiado / plazo */

df=df\
.withColumn('ratio_imp_fin_plazo', f.when((f.col('operation_term_month_number')==0), '')\
           .otherwise(f.col('risk_financed_amount')/f.col('operation_term_month_number')))

In [ ]:
# Comprobación de la variable /* Ratio financiacion - Propuesta negocio*/

df\
.select("risk_financed_amount", "operation_term_month_number", "ratio_imp_fin_plazo")\
.filter('operation_term_month_number <> 0').show(5, False)

### Indicador del plazo -  Agrupacion consensuada con negocio de la variable plazo_operacion_meses

In [ ]:
# Creación variable ### /* Indicador del plazo -  Agrupacion consensuada con negocio de la variable plazo_operacion_meses */

df=df\
.withColumn('ind_plazo', f.when((f.col('operation_term_month_number').isNull()), '')\
    .when((f.col('operation_term_month_number')<'24'), '1. Plazo < 24')\
    .when((f.col('operation_term_month_number')>='24') & (f.col('operation_term_month_number')<'36'), '2. 24 <= Plazo < 36')\
    .when((f.col('operation_term_month_number')>='36') & (f.col('operation_term_month_number')<'60'), '3. 36 <= Plazo < 60')\
    .when((f.col('operation_term_month_number')>='60') & (f.col('operation_term_month_number')<'72'), '4. 60 <= Plazo < 72')\
    .when((f.col('operation_term_month_number')>='72') & (f.col('operation_term_month_number')<'84'), '5. 72 <= Plazo < 84')\
    .otherwise('6. Plazo >= 84'))

In [ ]:
# Comprobración variable ### /* Indicador del plazo -  Agrupacion consensuada con negocio de la variable plazo_operacion_meses */ 

df\
.select("operation_term_month_number", "ind_plazo")\
.filter(df.operation_term_month_number=='83').show(20, False)

### Ind plazo 2

In [ ]:
# Creación variable ### /* Ind plazo 2 */

df=df\
.withColumn('ind_plazo2', f.when((f.col('ind_plazo')==''), '')\
    .when((f.col('operation_term_month_number')<='24'), 'CP')\
    .when((f.col('operation_term_month_number')>'24') & (f.col('operation_term_month_number')<='72'), 'MP')\
    .otherwise('LP'))

In [ ]:
# Comprobración variable ### /* Ind plazo 2 */

df\
.select("operation_term_month_number", "ind_plazo2")\
.filter(df.operation_term_month_number > '72').show(20, False)

### Ind plazo 3

In [ ]:
# Creación variable ### /* Ind plazo 3 */

df=df\
.withColumn('ind_plazo3', f.when((f.col('ind_plazo')==''), '')\
    .when((f.col('operation_term_month_number')<'36'), 'CP')\
    .when((f.col('operation_term_month_number')>='36') & (f.col('operation_term_month_number')<'72'), 'MP')\
    .otherwise('LP'))

In [ ]:
# Comprobración variable /* Ind plazo 3 */

df\
.select("operation_term_month_number", "ind_plazo3")\
.filter(df.operation_term_month_number == '72').show(20, False)

### Indicador de importe financiado - Agrupacion obtenida del EM autogrouping de la variable importe financiado 

In [ ]:
# Creación variable /* Indicador de importe financiado -

df=df\
.withColumn('ind_importe_finan', f.when((f.col('risk_financed_amount').isNull()), '')\
    .when((f.col('risk_financed_amount') < 11536), '1')\
    .when((f.col('risk_financed_amount') >= 11536) & (f.col('risk_financed_amount') <= 18095.93), '2')\
    .when((f.col('risk_financed_amount') >= 18095.83), '3')
    .otherwise(''))

In [ ]:
# Comprobración variable /* Indicador de importe financiado -

df\
.select("risk_financed_amount", "ind_importe_finan")\
    .filter(df.risk_financed_amount > 11536).show(20, False)

### Combinacion importe financiado - Nuevo usado

In [ ]:
# Creación variable /* Combinacion importe financiado - Nuevo usado */

df=df\
.withColumn('imp_finan_nuevousado', f.when((f.col('ind_importe_finan') == '') | (f.col('new_second_hand_type') == ''), '')\
    .otherwise(f.concat(f.col('ind_importe_finan').cast("string"), f.lit("/"),(f.col('new_second_hand_type').cast("string")))))

In [ ]:
# Comprobración variable /* Combinacion importe financiado - Nuevo usado */

df\
.select("ind_importe_finan", "new_second_hand_type", "imp_finan_nuevousado")\
    .show(20, False)

### Indicador edad - Agrupacion consensuada con negocio de la variable edad

In [ ]:
# Creación variable format ind_edad $20.;

df=df\
.withColumn('ind_edad', f.when((f.col('age_number').isNull()), 'MISSING')\
    .when((f.col('age_number')<'31'), '1. Edad < 31')\
    .when((f.col('age_number')>='31') & (f.col('age_number')<'36'), '2. 31 <= Edad < 36')\
    .when((f.col('age_number')>='36') & (f.col('age_number')<'42'), '3. 36 <= Edad < 42')\
    .when((f.col('age_number')>='42') & (f.col('age_number')<'45'), '4. 42 <= Edad < 45')\
    .when((f.col('age_number')>='45') & (f.col('age_number')<'50'), '5. 45 <= Edad < 50')\
    .otherwise('6. Edad >= 50'))

In [ ]:
# Comprobación de la condición format ind_edad $20.;

df\
.select("age_number", "ind_edad")\
.filter(df.age_number == '44').show(20, False)

In [ ]:
# Creación variable format ind_edad2 $20.;

df=df\
.withColumn('ind_edad2', f.when((f.col('age_number').isNull()), 'MISSING')\
    .when((f.col('age_number')<'42'), '1. Edad < 42')\
    .otherwise('2. Edad >= 42'))

In [ ]:
# Comprobación de la condición format ind_edad_2 $20.;

df\
.select("age_number", "ind_edad2")\
.filter(df.age_number == '72').show(20, False)

### anio_antiguedad_empresa, se corrige la variable antig_en_empresa a partir de la logica proporcionada por negocio

In [ ]:
# Creación variable /* anio_antiguedad_empresa, se corrige la variable antig_en_empresa a partir de la logica proporcionada por negocio */

df=df\
.withColumn('antiguedad_empresa',f.when(f.col("work_contract_type").isin(['1','4','9','A','B','C','D','G']),f.col("start_act_y_number"))\
        .otherwise(f.col("co_years_month_number")))

In [ ]:
# Comprobación /* anio_antiguedad_empresa, se corrige la variable antig_en_empresa a partir de la logica proporcionada por negocio */

df\
.select("work_contract_type", "start_act_y_number", "co_years_month_number", "antiguedad_empresa")\
.filter(df.work_contract_type == '2').show(20, False)

In [ ]:
# Creación variable anio_antiguedad_empresa

df=df\
.withColumn('anio_antiguedad_empresa',f.when(~(f.col("work_contract_type").isin(['1','4','9','A','B','C','D','G'])) & ((f.col("antiguedad_empresa") == 0) | (f.col("antiguedad_empresa") < 1950) | (f.col("antiguedad_empresa") > 2017)),-9999)\
    .when((f.col("work_contract_type").isin(['1','4','9','A','B','C','D','G'])) & ((f.col("start_act_y_number") < 1950) | (f.col("start_act_y_number")>2017)),-9999)\
    .when((f.col("work_contract_type").isin(['1','4','9','A','B','C','D','G'])) & (f.col("start_act_y_number").isin([0,-9999])),-9999)\
        .otherwise(f.year('request_date')-f.col('antiguedad_empresa'))) 

In [ ]:
# Comprobación variable anio_antiguedad_empresa

df\
.select("work_contract_type", "antiguedad_empresa", "request_date", "anio_antiguedad_empresa")\
.filter((df.work_contract_type == '2') & (df.antiguedad_empresa == 2018))\
.show(5, False)

### Indicador antigüedad en la empresa -  Consensuada con negocio

In [ ]:
# Creación variable /* Indicador antigüedad en la empresa - Consensuada con negocio */

df=df\
.withColumn('ind_ant_empresa',f.when(f.col("anio_antiguedad_empresa").isin(['',-9999]),'')\
    .when((f.col("anio_antiguedad_empresa") >= 0) & (f.col("anio_antiguedad_empresa") < 3),'1. 0 <= Antig_en_empresa < 3')\
    .when((f.col("anio_antiguedad_empresa") >= 3) & (f.col("anio_antiguedad_empresa") < 6),'2. 3 <= Antig_en_empresa < 6')\
    .when((f.col("anio_antiguedad_empresa") >= 6) & (f.col("anio_antiguedad_empresa") < 11),'3. 6 <= Antig_en_empresa < 11')\
    .when((f.col("anio_antiguedad_empresa") >= 11) & (f.col("anio_antiguedad_empresa") < 22),'4. 11 <= Antig_en_empresa < 22')\
        .otherwise('5. Antig_en_empresa >= 22'))  

In [ ]:
# Comprobación variable /* Indicador antigüedad en la empresa - Consensuada con negocio */

df\
.select("anio_antiguedad_empresa", "ind_ant_empresa")\
.filter((df.anio_antiguedad_empresa)==0)\
.show(5, False)

### Genero el ind_ant_empresa 2 para agrupar mas la antigüedad en la empresa

In [ ]:
# Creación variable /* Genero el ind_ant_empresa 2 para agrupar mas la antigüedad en la empresa */

df=df\
.withColumn('ind_ant_empresa2',f.when(f.col("ind_ant_empresa") == '', '')\
    .when((f.col("anio_antiguedad_empresa") >= 0) & (f.col("anio_antiguedad_empresa") < 3),'1. < 3 años')\
    .when((f.col("anio_antiguedad_empresa") >= 3) & (f.col("anio_antiguedad_empresa") < 11),'2. de 3 a 11 años')\
    .otherwise('3. más de 11 años'))

In [ ]:
# Comprobación variable /* Genero el ind_ant_empresa 2 para agrupar mas la antigüedad en la empresa */

df\
.select("anio_antiguedad_empresa", "ind_ant_empresa2")\
.filter((df.anio_antiguedad_empresa == '22'))\
.show(5, False)

### Genero el agrupado de tipo de contrato

In [ ]:
# Creación de variable /* Genero el agrupado de tipo de contrato */

df=df\
.withColumn('ind_tipo_contrato', f.when((f.col('work_contract_type').isin(['G','E','4','5','7','A','C'])),'SIN_INGRESOS/TEMPORERO')\
            .when((f.col('work_contract_type').isin(['9'])),'PENSIONISTA')\
            .when((f.col('work_contract_type').isin(['2'])),'FIJO')\
            .when((f.col('work_contract_type').isin(['1'])),'AUTONOMO')\
            .when((f.col('work_contract_type').isin(['3'])),'TEMPORAL')\
            .when((f.col('work_contract_type').isin(['6','B','D'])),'FUNCIONARIO/RENTISTA')
            .otherwise(''))

In [ ]:
# Comprobración varible /* Genero el agrupado de tipo de contrato */

df\
.select(df.work_contract_type, df.ind_tipo_contrato)\
.filter(df.work_contract_type == '6').show(5, False)

### Combinacion antigüedad en la empresa y tipo de contrato

In [ ]:
# Creación de variable /* Combinacion antigüedad en la empresa y tipo de contrato */

df=df\
.withColumn('ant_empresa_tip_trabajo', f.when((f.col('ind_ant_empresa') == '') | (f.col('work_contract_type') == ''),'')\
    .otherwise(f.concat(f.col('ind_ant_empresa').cast("string"), f.lit("/"),(f.col('work_contract_type').cast("string")))))

In [ ]:
# Comprobación de variable /* Combinacion antigüedad en la empresa y tipo de contrato */

df\
.select(df.ind_ant_empresa, df.work_contract_type, df.ant_empresa_tip_trabajo)\
.show(5, False)

### Combinacion antigüedad en la empresa y tipo de contrato  version 2

In [ ]:
# Creación de variable /* Combinacion antigüedad en la empresa y tipo de contrato version 2*/

df=df\
.withColumn('ant_empresa_tip_trabajo2', f.when((f.col('ind_ant_empresa2') == '') | (f.col('ind_tipo_contrato') == ''),'')\
    .otherwise(f.concat(f.col('ind_ant_empresa2').cast("string"), f.lit("/"),(f.col('ind_tipo_contrato').cast("string")))))

In [ ]:
# Comprobación de variable /* Combinacion antigüedad en la empresa y tipo de contrato version 2*/

df\
.select(df.ind_ant_empresa2, df.ind_tipo_contrato, df.ant_empresa_tip_trabajo2)\
.show(5, False)

### Combinacion ind edad y ind antiguedad en la empresa

In [ ]:
# Creación de variable /* Combinacion ind edad y ind antiguedad en la empresa */

df=df\
.withColumn('edad_ant_empresa', f.when((f.col('ind_edad') == '') | (f.col('ind_ant_empresa') == ''),'')\
    .otherwise(f.concat(f.col('ind_edad').cast("string"), f.lit("/"),(f.col('ind_ant_empresa').cast("string")))))

In [ ]:
# Comprobación de variable /* Combinacion ind edad y ind antiguedad en la empresa */

df\
.select(df.ind_edad, df.ind_ant_empresa, df.edad_ant_empresa)\
.show(5, False)

### Combinacion ind edad y ind antiguedad en la empresa version 2

In [ ]:
# Creación de variable /* Combinacion ind edad y ind antiguedad en la empresa version 2*/

df=df\
.withColumn('edad_ant_empresa2', f.when((f.col('ind_edad') == '') | (f.col('ind_ant_empresa2') == ''),'')\
    .otherwise(f.concat(f.col('ind_edad').cast("string"), f.lit("/"),(f.col('ind_ant_empresa2').cast("string")))))

In [ ]:
# Comprobación de variable /* Combinacion ind edad y ind antiguedad en la empresa version 2*/

df\
.select(df.ind_edad, df.ind_ant_empresa2, df.edad_ant_empresa2)\
.filter((df.ind_edad == '') | (df.ind_ant_empresa2 == '')).show(5, False)

### Combinacion ind edad y ind antiguedad en la empresa version 3

In [ ]:
# Creación de variable /* Combinacion ind edad y ind antiguedad en la empresa version 3*/

df=df\
.withColumn('edad_ant_empresa3', f.when((f.col('ind_edad2') == '') | (f.col('ind_ant_empresa2') == ''),'')\
    .otherwise(f.concat(f.col('ind_edad2').cast("string"), f.lit("/"),(f.col('ind_ant_empresa2').cast("string")))))

In [ ]:
# Comprobación de variable /* Combinacion ind edad y ind antiguedad en la empresa version 3*/

df\
.select(df.ind_edad2, df.ind_ant_empresa2, df.edad_ant_empresa3)\
.show(5, False)

### Estado civil / residencia habitual

In [ ]:
# Creación de la variable /* Estado civil / residencia habitual */

df=df\
.withColumn('Estaciv_reside', f.when((f.col('civil_status_type')=="") | (f.col('usual_residence_id').isNull()),'')\
           .otherwise(f.concat(f.col('civil_status_type').cast("string"), f.lit("/"),(f.col('usual_residence_id').cast("string")))))

In [ ]:
# Comprobración varible /* Estado civil / residencia habitual */

df\
.select(df.civil_status_type, df.usual_residence_id, df.Estaciv_reside)\
.show(5, False)

### Agrupado estado_civil 

In [ ]:
# Creación de variable /* Agrupado estado_civil */

df=df\
.withColumn('ag_estado_civil', f.when((f.col('civil_status_type')==''),'')\
            .when((f.col('civil_status_type')=='S'),'Soltero')\
            .when((f.col('civil_status_type').isin(['H','V','D','P'])),'SH-Viu-Dv-SJ')\
            .when((f.col('civil_status_type').isin(['C','N'])),'AM-Casado')\
            .otherwise('civil_status_type'))

In [ ]:
# Comprobración varible /* Agrupado estado_civil */

df\
.select(df.civil_status_type, df.ag_estado_civil)\
.filter(df.civil_status_type =='C').show(5, False)

### Agrupado residencia habitual

In [ ]:
# Creación de variable /* Agrupado residencia habitual */

df=df\
.withColumn('ag_reshabitual', f.when((f.col('usual_residence_id').isNull()),'')\
            .when((f.col('usual_residence_id').isin(['6','3'])),'PADRES_ARREN')\
            .when((f.col('usual_residence_id').isin(['4','5','7'])),'USU_OTROS_CONY')\
            .when((f.col('usual_residence_id').isin(['1','2'])),'LEAS_PROPIEDAD')\
            .otherwise('usual_residence_id'))

In [ ]:
# Comprobración varible /* Agrupado residencia habitual */

df\
.select(df.usual_residence_id, df.ag_reshabitual)\
.filter(df.usual_residence_id =='6').show(5, False)

### Estado civil / residencia habitual version 2

In [ ]:
# Creación variable /* Estado civil / residencia habitual version 2*/

df=df\
.withColumn('Estaciv_reside2', f.when((f.col('ag_estado_civil')=='') | (f.col('ag_reshabitual')==''), '')\
    .otherwise(f.concat(f.col('ag_estado_civil').cast("string"), f.lit("/"),(f.col('ag_reshabitual').cast("string")))))

In [ ]:
# Creación variable /* Estado civil / residencia habitual version 2*/

df\
.select(df.ag_estado_civil, df.ag_reshabitual, df.Estaciv_reside2)\
.show(5, False)

### Combinacion edad residencia habitual

In [ ]:
# Creación variable /* Combinacion edad residencia habitual */

df=df\
.withColumn('edad_res_habitual',(f.concat(f.col('ind_edad').cast("string"), f.lit("/"),(f.col('ag_reshabitual').cast("string")))))

In [ ]:
# Comprobación variable /* Estado civil / residencia habitual version 2*/

df\
.select(df.ind_edad, df.ag_reshabitual, df.edad_res_habitual)\
.show(5, False)

### Combinacion edad residencia habitual version 2 

In [ ]:
# Creación variable /* Combinacion edad residencia habitual version 2 */

df=df\
.withColumn('edad_res_habitual2',(f.concat(f.col('ind_edad2').cast("string"), f.lit("/"),(f.col('ag_reshabitual').cast("string")))))

In [ ]:
# Creación variable /* Combinacion edad residencia habitual version 2 */

df\
.select(df.ind_edad2, df.ag_reshabitual, df.edad_res_habitual2)\
.show(5, False)

### Total ingresos

In [ ]:
# Creación de la variable /* Total ingresos */

df=df\
.withColumn('Total_ingresos',(f.col("fix_income_amount"))+(f.col("dem_var_inc_amount")))

In [ ]:
# Comprobación de la variable /* Total ingresos */

df\
.select(df.fix_income_amount, df.dem_var_inc_amount, df.Total_ingresos)\
.show(5, False)

### Ratio ingresos / cuota

In [ ]:
# Creación de la variable /* Ratio ingresos / cuota */

df=df\
.withColumn('Ratio_Ingresos_cuota',f.when((f.col("risk_fee_amount")=='0'),'0')
        .otherwise(f.col("Total_ingresos")/f.col("risk_fee_amount")))

In [ ]:
# Comprobación de la variable /* Ratio ingresos / cuota */

df\
.select(df.risk_fee_amount, df.Total_ingresos, df.Ratio_Ingresos_cuota)\
.filter(df.risk_fee_amount != '0').show(5, False)

### Ratio disponible dispuesto

In [ ]:
# Creación de la variable * Ratio disponible dispuesto *;

df=df\
.withColumn('Ratio_disponible_dispuesto', f.col('prop_lim_operation_amount')/f.col('risk_financed_amount'))

In [ ]:
# Comprobación de la variable * Ratio disponible dispuesto *;

df\
.select(df.prop_lim_operation_amount, df.risk_financed_amount, df.Ratio_disponible_dispuesto)\
.show(5, False)

### Ratio deuda prestamo respecto total prestado

In [ ]:
# Creación de la variable * Ratio deuda prestamo respecto total prestado *;

df=df\
.withColumn('Ratio_prestamo_riesgo', f.col('risk_financed_amount')/(f.col('loan_lv_rk_amount')+f.col('risk_financed_amount')))

In [ ]:
# Comprobación de la variable * Ratio deuda prestamo respecto total prestado *;

df\
.select(df.risk_financed_amount, df.loan_lv_rk_amount, df.Ratio_prestamo_riesgo)\
.show(5, False)

### Cuota BBVA

In [ ]:
# Creación de la variable * Cuota BBVA *;

df=df\
.withColumn('Cuota_bbva', ((f.col('risk_financed_amount')+f.col('loan_lv_rk_amount'))/f.col('operation_term_month_number')))

In [ ]:
# Comprobación de la variable * Cuota BBVA *;

df\
.select(df.risk_financed_amount, df.loan_lv_rk_amount, df.operation_term_month_number, df.Cuota_bbva)\
.show(5, False)

### Numero de vueltas

In [ ]:
# Creación de la variable * Numero de vueltas *;

df=df\
.withColumn('Numero_de_vueltas',f.when((f.col("risk_fee_amount")=='0'),'0')
        .otherwise(f.col("Total_ingresos")/f.col("risk_fee_amount")))

In [ ]:
# Comprobación de la variable * Numero de vueltas *;

df\
.select(df.risk_fee_amount, df.Total_ingresos, df.Numero_de_vueltas)\
.filter(df.risk_fee_amount == '0').show(5, False)

### Numero de vueltas BBVA

In [ ]:
# Creación de la variable * Numero de vueltas BBVA*;

df=df\
.withColumn('Num_vueltas_BBVA',f.when((f.col("Cuota_bbva")=='0'),'0')
        .otherwise(f.col("Total_ingresos")/f.col("risk_fee_amount")))

In [ ]:
# Comprobación de la variable * Numero de vueltas BBVA*;

df\
.select(df.risk_fee_amount, df.Cuota_bbva, df.Total_ingresos, df.Num_vueltas_BBVA)\
.filter(df.Cuota_bbva == '0').show(5, False)

### Tasa de Esfuerzo

In [ ]:
# Creación de la variable * Tasa de Esfuerzo *;

df=df\
.withColumn('Tasa_de_esfuerzo',f.when((f.col("Total_ingresos")=='0'),'0')
        .otherwise(f.col("risk_fee_amount")/f.col("Total_ingresos")))

In [ ]:
# Comprobación de la variable * Tasa de Esfuerzo *;

df\
.select(df.Total_ingresos, df.risk_fee_amount, df.Tasa_de_esfuerzo)\
.filter(df.Total_ingresos != '0').show(5, False)

### Indicador esfuerzo - plazo

In [ ]:
# Creación de variable * Indicador esfuerzo - plazo *;

df=df\
.withColumn('Ind_esfuerzo_plazo', f.when((f.col('operation_term_month_number')<=48) & (f.col('Tasa_de_esfuerzo')<=0.15),'01. <=48meses y esfuerzo <= 15%')\
            .when((f.col('operation_term_month_number')>48) & (f.col('Tasa_de_esfuerzo')<=0.15),'02. >48meses y esfuerzo <= 15%')\
            .when((f.col('operation_term_month_number')<=48) & (f.col('Tasa_de_esfuerzo')>0.15),'03. <=48meses y esfuerzo > 15%')\
            .when((f.col('operation_term_month_number')>48) & (f.col('Tasa_de_esfuerzo')>0.15),'04. >48meses y esfuerzo > 15%')\
            .otherwise(''))

In [ ]:
# Comprobación de la variable * Indicador esfuerzo - plazo *;

df\
.select(df.operation_term_month_number, df.Tasa_de_esfuerzo, df.Ind_esfuerzo_plazo)\
.filter((df.operation_term_month_number <= 48) & (df.operation_term_month_number <= 0.15))\
.show(5, False)

### Indicador esfuerzo - plazo version 2

In [ ]:
# Creación de variable * Indicador esfuerzo - plazo version 2 *;

df=df\
.withColumn('Ind_esfuerzo_plazo2', f.when((f.col('ind_plazo2')=='CP') & (f.col('Tasa_de_esfuerzo')<=0.15),'01. Corto Plazo y esfuerzo <= 15%')\
            .when((f.col('ind_plazo2')=='CP') & (f.col('Tasa_de_esfuerzo')>0.15),'02. Corto Plazo y esfuerzo > 15%')\
            .when((f.col('ind_plazo2')=='MP') & (f.col('Tasa_de_esfuerzo')<=0.15),'03. Medio Plazo y esfuerzo <= 15%')\
            .when((f.col('ind_plazo2')=='MP') & (f.col('Tasa_de_esfuerzo')>0.15),'04. Medio Plazo y esfuerzo > 15%')\
            .when((f.col('ind_plazo2')=='LP') & (f.col('Tasa_de_esfuerzo')<=0.15),'05. Largo Plazo y esfuerzo <= 15%')\
            .when((f.col('ind_plazo2')=='LP') & (f.col('Tasa_de_esfuerzo')>0.15),'06. Largo Plazo y esfuerzo > 15%')\
            .otherwise(''))

In [ ]:
# Comprobación de la variable * Indicador esfuerzo - plazo version 2 *;

df\
.select(df.ind_plazo2, df.Tasa_de_esfuerzo, df.Ind_esfuerzo_plazo2)\
.filter((df.ind_plazo2 == 'CP') & (df.Tasa_de_esfuerzo <= 0.15)).show(5, False)

### Estado civil - Regimen matrimonial

In [ ]:
# Creación de la variable /* Estado civil - Regimen matrimonial */

df=df\
.withColumn('estadociv_regmat',f.when((f.col("civil_status_type")=='C'), (f.concat(f.col('civil_status_type').cast("string"), f.lit("/"),(f.col('marital_reg_type').cast("string")))))\
        .otherwise(f.concat(f.col('civil_status_type'), f.lit("/N"))))

In [ ]:
# Comprobación de la variable /* Estado civil - Regimen matrimonial */

df\
.select(df.civil_status_type, df.marital_reg_type, df.estadociv_regmat)\
.filter(df.civil_status_type != 'C').show(5, False)

### Alerta incidencias

In [ ]:
# Creación de la variable /* Alerta incidencias */

df=df\
.withColumn('alerta_inc',f.when((f.col("asnef_issue_number") > '0') | (f.col("irr_inv_d_number") > '0'),'1')
        .otherwise('0'))

In [ ]:
# Comprobación de la variable /* Estado civil - Regimen matrimonial */

df\
.select(df.asnef_issue_number, df.irr_inv_d_number, df.alerta_inc)\
.filter(df.asnef_issue_number == '0').show(5, False)

### Plazo en meses - Destino operacion

In [ ]:
# Creación de la variable /* Plazo en meses - Destino operacion */

df=df\
.withColumn('plazo_destino', (f.concat(f.col('ind_plazo').cast("string"), f.lit("_"),(f.col('ppty_dest_id').cast("string")))))

In [ ]:
# Creación de la variable /* Plazo en meses - Destino operacion 2*/

df=df\
.withColumn('plazo_destino2', (f.concat(f.col('ind_plazo2').cast("string"), f.lit("_"),(f.col('ppty_dest_id').cast("string")))))

In [ ]:
# Creación de la variable /* Plazo en meses - Destino operacion 3*/

df=df\
.withColumn('plazo_destino3', (f.concat(f.col('ind_plazo3').cast("string"), f.lit("_"),(f.col('ppty_dest_id').cast("string")))))

In [ ]:
# Comprobación de la variable /* Plazo en meses - Destino operacion */

df\
.select(df.ind_plazo, df.ppty_dest_id, df.plazo_destino)\
.show(5, False)

In [ ]:
# Comprobación de la variable /* Plazo en meses - Destino operacion 2*/

df\
.select(df.ind_plazo, df.ppty_dest_id, df.plazo_destino2)\
.show(5, False)

In [ ]:
# Comprobación de la variable /* Plazo en meses - Destino operacion 2*/
df\
.select(df.ind_plazo, df.ppty_dest_id, df.plazo_destino3)\
.show(5, False)

### Se genera la variable comunidad autonoma

In [ ]:
### Creación de la variable CD_PROVINCIA

df=df\
.withColumn('CD_PROVINCIA', (f.substring(f.trim(f.col('fisc_resid_prov_id')), 1, 2)))

In [ ]:
### Comprobación resultado de la variable CD_PROVINCIA

df\
.select(df.fisc_resid_prov_id, df.CD_PROVINCIA)\
.show(5, False)

In [ ]:
# Creación de variable /* Genero la variable comunidad autonoma */

df=df\
.withColumn('COM_AUTONOMA', f.when((f.col('CD_PROVINCIA').isin(['04','18','11','41','29','23','14','21'])),'Andalucia')\
    .when((f.col('CD_PROVINCIA').isin(['50','22','44'])),'Aragon')\
    .when((f.col('CD_PROVINCIA').isin(['33'])),'Asturias')\
    .when((f.col('CD_PROVINCIA').isin(['07'])),'Baleares')\
    .when((f.col('CD_PROVINCIA').isin(['38','35'])),'Canarias')\
    .when((f.col('CD_PROVINCIA').isin(['39'])),'Cantabria')\
    .when((f.col('CD_PROVINCIA').isin(['37','24','40','34','49','09','42','47','05'])),'Castilla_y_Leon')\
    .when((f.col('CD_PROVINCIA').isin(['43','25','08','17'])),'Cataluña')\
    .when((f.col('CD_PROVINCIA').isin(['51'])),'Ceuta')\
    .when((f.col('CD_PROVINCIA').isin(['06','10'])),'Extremadura')\
    .when((f.col('CD_PROVINCIA').isin(['15','36','27','32'])),'Galicia')\
    .when((f.col('CD_PROVINCIA').isin(['45','19','13','16','02'])),'La_Mancha')\
    .when((f.col('CD_PROVINCIA').isin(['26'])),'La Rioja')\
    .when((f.col('CD_PROVINCIA').isin(['28'])),'Madrid')\
    .when((f.col('CD_PROVINCIA').isin(['52'])),'Melilla')\
    .when((f.col('CD_PROVINCIA').isin(['30'])),'Murcia')\
    .when((f.col('CD_PROVINCIA').isin(['31'])),'Navarra')\
    .when((f.col('CD_PROVINCIA').isin(['20','48','01'])),'Pais_Vasco')\
    .when((f.col('CD_PROVINCIA').isin(['12','46','03'])),'Valenciana')\
    .when((f.col('CD_PROVINCIA').isin(['00'])),'00')\
        .otherwise(''))

In [ ]:
### Comprobación resultado de la variable CD_PROVINCIA

df\
.select(df.fisc_resid_prov_id, df.CD_PROVINCIA, df.COM_AUTONOMA)\
.show(5, False)

## ESCRITURA Y LECTURA FICHERO NUEVAS VARIABLES

In [ ]:
#Creamos una vista del fichero

df.createOrReplaceTempView("df")

In [ ]:
# Comprobamos que la vista se ha creado satisfactoriamente

spark.sql("""
SELECT * FROM df
""").show(5, False)

In [ ]:
# Comprobamos que la vista se ha creado satisfactoriamente

df_gv4=spark.sql("""
SELECT * FROM df
""")

In [ ]:
# Persistimos el fichero en el Sandbox, con el nombre df_gv4

spark.sql("""
CREATE TABLE df_gv4
USING PARQUET
LOCATION '/data/sandboxes/...'
AS
SELECT *
FROM df
""")

In [ ]:
# Guardamos en la variable df_gv4_sas, los campos necesarios

df_gv4_sas=spark.sql("""
SELECT work_contract_type, 
Tipo_cliente, 
cnae_cno, 
cnae_cno_id, 
company_cnae_id, 
ppty_dest_id, ppty_subdest_id,
Destino_subdestino, 
ppal_good_finance_amount, 
risk_financed_amount, 
ratio_financiacion, 
prop_lim_operation_amount, 
ratio_financiacion_2, 
operation_term_month_number, 
ratio_imp_fin_plazo, 
ind_plazo, 
ind_plazo2, 
ind_plazo3, 
ind_importe_finan, 
new_second_hand_type, 
imp_finan_nuevousado, 
age_number, 
ind_edad, 
ind_edad2, 
start_act_y_number, 
co_years_month_number, 
antiguedad_empresa, 
request_date, 
anio_antiguedad_empresa, 
ind_ant_empresa, 
ind_ant_empresa2, 
ant_empresa_tip_trabajo, 
ind_tipo_contrato, 
ant_empresa_tip_trabajo2, 
edad_ant_empresa, 
edad_ant_empresa2, 
edad_ant_empresa3, 
civil_status_type, 
usual_residence_id, 
Estaciv_reside, 
ag_estado_civil, 
ag_reshabitual, 
Estaciv_reside2, 
edad_res_habitual, 
edad_res_habitual2, 
fix_income_amount, 
dem_var_inc_amount, 
Total_ingresos,
risk_fee_amount,
Ratio_Ingresos_cuota, 
Ratio_disponible_dispuesto, 
loan_lv_rk_amount, 
Ratio_prestamo_riesgo, 
Cuota_bbva, 
Numero_de_vueltas, 
Num_vueltas_BBVA, 
Tasa_de_esfuerzo, 
Ind_esfuerzo_plazo, 
Ind_esfuerzo_plazo2, 
marital_reg_type, 
estadociv_regmat, 
asnef_issue_number, 
irr_inv_d_number, 
alerta_inc, 
plazo_destino, 
plazo_destino2, 
plazo_destino3, 
fisc_resid_prov_id, 
CD_PROVINCIA, 
COM_AUTONOMA
FROM df_gv4
""")

In [ ]:
# Creamos una vista de la select anterior

df_gv4_sas.createOrReplaceTempView("df_gv4_sas")

In [ ]:
# Persistimos el fichero en el Sandbox, con el nombre df_gv4_sas

spark.sql("""
CREATE TABLE df_gv4_sas
USING PARQUET
LOCATION '/data/sandboxes/...'
AS
SELECT *
FROM df_gv4_sas
""")

In [ ]:
# Comprobamos que la vista df_gv4_sas se ha creado correctamente

spark.sql("""
SELECT *
FROM df_gv4_sas
""").show(5, False)